In [3]:
import random
from firebase_admin import initialize_app, credentials, db

In [1]:
import json

In [4]:
cred = credentials.Certificate('creds.json')

In [5]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [6]:
seed = random.randrange(2 ** 32)
seed

3489313612

In [7]:
players = ["A", "B", "C"]
random.shuffle(players)

In [9]:
with open("board.json") as json_file:
    board = json.load(json_file)

In [10]:
db.reference("test/initial").set({
    'seed': seed,
    'players': players,
    'board': board,
})